In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import cv2

import maxflow

from frame_utils import *
from visualization_utils import *
from video_utils import * 
from video_processing import *

from VidFrame import *

from tqdm.notebook import tqdm

from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float, img_as_ubyte
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

%load_ext autoreload
%autoreload 2

### Image Pre-Processing (Optional)

In [ ]:
H = 1080 // 3
W = 1920 // 3

vid_frames = load_video('videos/1_Basketball.mp4', H, W)
print(vid_frames.shape)

In [ ]:
n_clusters = 500
compactness = 20

In [ ]:
fig = segment_frames(vid_frames, [0,100,200,300], 2, 2, n_clusters, compactness)
plt.show()

In [ ]:
vid_labels = segment_video(vid_frames, tqdm, n_clusters, compactness)
print(vid_labels.shape)

In [ ]:
np.save('preprocess_cache/1_Basketball.npy', vid_labels)

In [ ]:
# vidwrite_from_numpy('outputs/1_Basketball.mp4', vid_frames)

### Visualizing Segmented Frames

In [ ]:
H = 1080 // 3
W = 1920 // 3

vid_frames = load_video('videos/1_Basketball.mp4', H, W)[:100]
label_frames = np.load('preprocess_cache/1_Basketball.npy')

In [ ]:
F, H, W, C = vid_frames.shape

segmented_vid_frames = np.zeros_like(vid_frames)

for f in tqdm(range(F)):
    segmented_vid_frames[f] = get_boundary_segments(vid_frames[f], label_frames[f])

# vidwrite_from_numpy('outputs/1_Baketball_Segmented.mp4', segmented_vid_frames)

In [ ]:
plt.figure()

fig = visualize_images(segmented_vid_frames[80:84], 2, 2)

plt.show()

In [ ]:
VidFrame80 = VidFrame(vid_frames[80], label_frames[80])
VidFrame81 = VidFrame(vid_frames[81], label_frames[81])

In [ ]:
jaccard_threshold = 0.3

mpixels = VidFrame81.calculateMotionSuperpixels(VidFrame80, jaccard_threshold)

print(mpixels)

In [ ]:
imgs = np.zeros((3, H, W, 3), dtype=VidFrame81.vid_frame.dtype)

imgs[0] = VidFrame80.getSegmentedImage()
imgs[1] = VidFrame81.getSegmentedImage()

imgs[2] = VidFrame81.colorMotionSuperpixels()

fig = visualize_images(imgs, 1, 3)
plt.show()

In [ ]:
N_background_segments = 25
background_compactness = 10

In [ ]:
view_vidframe_segmentations(VidFrame81, N_background_segments, background_compactness)

### Fit Gaussian Mixture Models for Background and Foreground

In [ ]:
b_mogs, f_mogs, N_b, N_f = VidFrame81.generateGaussians(3)

t_k = np.zeros(N_f)

priors_b = calculate_background_priors(b_mogs)
priors_f = calculate_foreground_priors(f_mogs, N_f)

In [ ]:
VidFrame81.createMaxflowGraph(b_mogs, priors_b, f_mogs, priors_f, 500)

maxflow_val,object_mask = VidFrame81.createObjectSegmentation()

print(maxflow_val)
plt.figure()
plt.imshow(object_mask, cmap='gray')

In [ ]:
VidFrame82 = VidFrame(vid_frames[82], label_frames[82])

mpixels = VidFrame82.calculateMotionSuperpixels(VidFrame81, jaccard_threshold)

print(mpixels)

In [ ]:
view_vidframe_segmentations(VidFrame82, N_background_segments, background_compactness)

In [ ]:
VidFrame82.createMaxflowGraph(b_mogs, priors_b, f_mogs, priors_f, 500)

maxflow_val,object_mask = VidFrame82.createObjectSegmentation()

print(maxflow_val)
plt.figure()
plt.imshow(object_mask, cmap='gray')

In [ ]:
update_background_models(VidFrame82, b_mogs)

In [ ]:
update_foreground_models(VidFrame82, f_mogs, 0.8, 15)

N_f = len(f_mogs)

priors_b = calculate_background_priors(b_mogs)
priors_f = calculate_foreground_priors(f_mogs, N_f)

In [ ]:
VidFrame83 = VidFrame(vid_frames[83], label_frames[83])

mpixels = VidFrame83.calculateMotionSuperpixels(VidFrame82, jaccard_threshold)

print(mpixels)

view_vidframe_segmentations(VidFrame83, N_background_segments, background_compactness)

VidFrame83.createMaxflowGraph(b_mogs, priors_b, f_mogs, priors_f, 500)

maxflow_val,object_mask = VidFrame83.createObjectSegmentation()

print(maxflow_val)
plt.figure()
plt.imshow(object_mask, cmap='gray')

### Process multiple frames at once

In [ ]:
N_background_segments = 25
background_compactness = 10

In [ ]:
VidFrame60 = VidFrame(vid_frames[60], label_frames[60])
VidFrame61 = VidFrame(vid_frames[61], label_frames[61])

mpixels = VidFrame61.calculateMotionSuperpixels(VidFrame60, jaccard_threshold)

print(mpixels)

view_vidframe_segmentations(VidFrame61, N_background_segments, background_compactness)

b_mogs, f_mogs, N_b, N_f = VidFrame61.generateGaussians(3)

t_k = np.zeros(N_f)

priors_b = calculate_background_priors(b_mogs)
priors_f = calculate_foreground_priors(f_mogs, N_f)


VidFrame61.createMaxflowGraph(b_mogs, priors_b, f_mogs, priors_f, 500)

maxflow_val,object_mask = VidFrame61.createObjectSegmentation()

print(maxflow_val)
plt.figure()
plt.imshow(object_mask, cmap='gray')

In [ ]:
object_segmentations = process_frames(vid_frames, label_frames, 60, 70)

In [ ]:
visualize_images(object_segmentations, nrows=5, ncols=2)

In [ ]:
background_frame = vid_frames[0]
H, W, C = background_frame.shape

obj_removed_frames = np.zeros((10, H, W, C), dtype=np.uint8)

for i in range(10):
    plt.figure()
    
    obj_removed_frames[i] = img_as_ubyte(mark_boundaries(vid_frames[60+i], object_segmentations[i].astype(np.uint32)))
    
    plt.imshow(obj_removed_frames[i])